Planetary Nebulae Project (Part 2)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.fft import fft, fftfreq
import matplotlib.patches as mpatches
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
import io

In [25]:
# Fermi Database (Gamma-Ray) ---------------------------------------------------
hdul = fits.open('Resources/gll_psc_v35.fit')
data = hdul[1].data

o3 = [row[4] for row in data]
a3 = [row[5] for row in data]

go3 = [float(value) for value in o3]
ga3 = [float(value) for value in a3]

glon3 = go3 * u.deg
glat3 = ga3 * u.deg

g_coords3 = SkyCoord(l=glon3,b=glat3, frame='galactic')

In [26]:
# Chandra Database (X-Ray) -----------------------------------------------------
dtypee = [('blank', 'f8'),('name', 'U20'), ('ra', 'U15'), ('dec', 'U15'), ('lii', 'f8'), ('bii', 'f8'), ('error_ellipse_r0', 'f8'), ('error_ellipse_r1', 'f8'), ('error_ellipse_angle', 'f8')]
data4 = np.genfromtxt('Resources/chandra.txt', delimiter='|', skip_header=3, dtype=dtypee, invalid_raise=False)

go4 = data4['lii']
ga4 = data4['bii']
glon4 = go4 * u.deg
glat4 = ga4 * u.deg

g_coords4 = SkyCoord(l=glon4,b=glat4, frame='galactic')

In [27]:
# Swift Database (Ultraviolet & Optical) ---------------------------------------
dtypeee = [('blank', 'f8'),('source_number', 'i8'), ('name', 'U25'), ('ra', 'U15'), ('dec', 'U15'), ('lii', 'f8'), ('bii', 'f8'), ('error_radius', 'f8'), ('count_rate_fb', 'f8'), ('count_rate_fb_pos_err', 'f8'), ('count_rate_fb_neg_err', 'f8'), ('pow_flux_a', 'f8'), ('pow_flux_a_pos_err', 'f8'), ('pow_flux_a_neg_err', 'f8'), ('apec_flux_a', 'f8'), ('apec_flux_a_pos_err', 'f8'), ('apec_flux_a_neg_err', 'f8')]
data5 = np.genfromtxt('Resources/swift.txt', delimiter='|', skip_header=3, dtype=dtypeee, invalid_raise=False)

go5 = data5['lii']
ga5 = data5['bii']

glon5 = go5 * u.deg
glat5 = ga5 * u.deg

g_coords5 = SkyCoord(l=glon5,b=glat5, frame='galactic')

In [29]:
dtype6 = [('Name', 'U50'),('RA (J2000)', 'U12'),('Dec (J2000)', 'U12')]
data6 = np.genfromtxt('Resources/notable_data.csv', delimiter=',',dtype=dtype6,names=True,encoding='utf-8')

ra6 = data6['RA_J2000']
dec6 = data6['Dec_J2000']

coords = SkyCoord(ra=ra6, dec=dec6, unit=(u.hourangle, u.deg), frame='icrs')
galactic_coords = coords.galactic

In [38]:
################################################################################
#                                 Fermi Matrix                                 #
################################################################################

# Matrix of Distances -------------------------------------------------------------
fd = np.zeros((len(galactic_coords), len(g_coords3)))
fd = galactic_coords[:, None].separation(g_coords3[None, :]).deg

# Shortest Sources and Distances -----------------------------------------------
with fits.open('Resources/gll_psc_v35.fit') as hdul:
    fermi_data = hdul[1].data
    fermi_names = fermi_data.columns[0].array

closest_indices = np.argmin(fd, axis=1)
closest_distances = np.min(fd, axis=1)

for i, (index, distance) in enumerate(zip(closest_indices, closest_distances)):
    fermi_name = fermi_names[index].decode('utf-8').strip()
    print(f"PNe {i}: Closest Fermi signal is '{fermi_name}' with distance {distance:.4f} degrees")

PNe 0: Closest Fermi signal is '4FGL J2003.6+2333' with distance 1.2478 degrees
PNe 1: Closest Fermi signal is '4FGL J1848.5+3243' with distance 1.0917 degrees
PNe 2: Closest Fermi signal is '4FGL J0140.4+5114' with distance 0.4405 degrees
PNe 3: Closest Fermi signal is '4FGL J1758.2+6532' with distance 1.4693 degrees
PNe 4: Closest Fermi signal is '4FGL J1537.3-6110' with distance 0.4028 degrees
PNe 5: Closest Fermi signal is '4FGL J1822.8-1118' with distance 0.4148 degrees
PNe 6: Closest Fermi signal is '4FGL J1818.5-2036' with distance 1.3032 degrees
PNe 7: Closest Fermi signal is '4FGL J0537.5+0959' with distance 1.4468 degrees
PNe 8: Closest Fermi signal is '4FGL J1717.0-3218' with distance 0.2910 degrees
PNe 9: Closest Fermi signal is '4FGL J1746.0-2349' with distance 0.4867 degrees
PNe 10: Closest Fermi signal is '4FGL J1701.9-4625' with distance 0.5663 degrees
PNe 11: Closest Fermi signal is '4FGL J1726.7-3930' with distance 0.4476 degrees
PNe 12: Closest Fermi signal is '4FGL 

In [ ]:
################################################################################
#                                Chandra Matrix                                #
################################################################################

# Matrix of Distances -------------------------------------------------------------
cd = np.zeros((len(galactic_coords), len(g_coords4)))
cd = galactic_coords[:, None].separation(g_coords4[None, :]).deg
print(cd)
# Shortest Sources and Distances -----------------------------------------------
shortest_distances = np.min(cd, axis=1)
closest_sources = np.argmin(cd, axis=1)

[[60.16792229 60.17366762 59.74525193 ... 64.07688914 59.75370393
  78.15949292]
 [74.79389972 55.13338041 74.15640216 ... 80.16887195 74.16993644
  95.97959683]
 [46.21405931 23.80307355 45.07565759 ... 55.60212691 45.10197224
  82.39781593]
 ...
 [80.37531607 89.38486676 80.46731027 ... 79.7944931  80.46406331
  79.84408374]
 [76.95802527 84.29407848 76.9646379  ... 77.1311633  76.96331594
  79.64771502]
 [76.85854464 84.21272633 76.86426939 ... 77.04065314 76.86296714
  79.59596655]]
